# Telecoms Classifier

We need to get all (or a subset initially) of the Cordis dataset and GtR dataset to create our training and test data.

Training data will be the Cordis dataset, labelled according to whether the topic is "telecoms" or not. 

We first need to access the data using the database.



Do I have the right version of weaviate installed? 

https://github.com/langchain-ai/langchain/issues/18809

In [1]:
import pandas as pd
import weaviate
import weaviate.classes as wvc
import weaviate.classes.query as wvcq
from weaviate.exceptions import WeaviateQueryError
import pprint as pprint

import time
from dateutil import parser
from datetime import datetime, timezone

from helper_functions import *
import re


In [2]:
keyword_include = ["5G",
    "6G",
    "RF",
    "Radio Frequency",
    "Radio-Frequency",
    "Base station",
    "MIMO",
    "GHz",
    "Massive MIMO",
    "4G",
    "Terahertz",
    "Digital Signal Processing",
    "Air Interface",
    "Backhaul",
    "Radio Access Network",
    "RAN",
    "MHz",
    "Vehicular Communications",
    "Reconfigurable Intelligent Surfaces",
    "RIS",
    "mmWave",
    "Multiple-Input Multiple-Output",
    "3G",
    "Beamforming",
    "Cognitive Radio",
    "Channel Model",
    "LiFi",
    "Channel estimation",
    "LTE",
    "Core network",
    "Millimeter wave",
    "Wi-Fi",
    "New Radio",
    "DSP",
    "Fronthaul",
    "Line Of Sight",
    "v2x",
    "O-RAN",
    "Vehicle-to-vehicle",
    "Network management",
    "Open RAN"]


keyword_exclude = ["MRI",
    "Satellites",
    "Non terrestrial communications",
    "Non-terrestrial communications",
    "Drones",
    "Semiconductors",
    "Telescope",
    "Cancer",
    "DNA",
    "Drug discovery",
    "Neurobiology",
    "Nuclear magnetic resonance",
    "NMR",
    "Wastewater treatment processes",
    "sustainable agriculture",
    "public health",
    "economics",
    "ecosystems",
    "drinking water treatment processes",
    "composites",
    "business models",
    "bacteriology",
    "National Oceanography Centre",
    "aircraft"]

semantic_include = [
    "Telecoms",
    "Telecommunications",
    "Automatic vehicles"
]

semantic_exclude = [
    "Astronomy",
    "Medicine",
    "Medical",
    "Climate change",
    "Oceanography"
]
search = []
# for x in ["keyword_include","keyword_exclude","semantic_include","semantic_exclude"]:
autocut = {'keyword_exclude' : 1000, 'semantic_exclude' : 1000}

for x in ["semantic_include","semantic_exclude","keyword_include","keyword_exclude"]:

    exec(f"search = search +  list(map(lambda e: (e,'keyword' in '{x}', ['{x}'], autocut[x] if 'exclude' in '{x}' else None,), {x}))")


search


[('Telecoms', False, ['semantic_include'], None),
 ('Telecommunications', False, ['semantic_include'], None),
 ('Automatic vehicles', False, ['semantic_include'], None),
 ('Astronomy', False, ['semantic_exclude'], 1000),
 ('Medicine', False, ['semantic_exclude'], 1000),
 ('Medical', False, ['semantic_exclude'], 1000),
 ('Climate change', False, ['semantic_exclude'], 1000),
 ('Oceanography', False, ['semantic_exclude'], 1000),
 ('5G', True, ['keyword_include'], None),
 ('6G', True, ['keyword_include'], None),
 ('RF', True, ['keyword_include'], None),
 ('Radio Frequency', True, ['keyword_include'], None),
 ('Radio-Frequency', True, ['keyword_include'], None),
 ('Base station', True, ['keyword_include'], None),
 ('MIMO', True, ['keyword_include'], None),
 ('GHz', True, ['keyword_include'], None),
 ('Massive MIMO', True, ['keyword_include'], None),
 ('4G', True, ['keyword_include'], None),
 ('Terahertz', True, ['keyword_include'], None),
 ('Digital Signal Processing', True, ['keyword_inclu

In [3]:
client = weaviate.connect_to_local(host="10.68.169.51",additional_config=weaviate.config.AdditionalConfig(timeout=(30, 3000)))

In [4]:
df_data = grab_df("telecoms")


In [5]:
df_data.head()

,description,endDate,fundAmount,funder,url,status,name,fundAmountUK,originalID,dataSource,...,fundCurrency,grantCategory,startDate,fundScheme,uuid,Lead Organisation,Participant Organisation(s),score,metadata,concept
0,The smart lamp columns will have enhanced foun...,2025-03-31 12:00:00,192000,None,https://uktin.net/navigate-uk-telecoms/governm...,Open,Westminster City Council,NaN,None,UKTIN,...,GBP,Government Funded,2023-10-02 12:00:00,Smart Infrastructure Pilots Programme,d66e5a5d-dbdc-5e09-81b4-5cde101a22ff,,,3.619544,"MetadataReturn(creation_time=None, last_update...",telecoms
1,The multi-purpose smart columns will be instal...,2025-03-31 12:00:00,241500,None,https://uktin.net/navigate-uk-telecoms/governm...,Open,Tees Valley Combined Authority,NaN,None,UKTIN,...,GBP,Government Funded,2023-10-02 12:00:00,Smart Infrastructure Pilots Programme,97a259e4-7726-5c0d-8e92-871fc8e3e4aa,Tees Valley Combined Authority,Tees Valley Combined Authority,3.619544,"MetadataReturn(creation_time=None, last_update...",telecoms
2,The Dorset Open Network Ecosystem (DONE) build...,2025-03-31 12:00:00,3615974,None,https://uktin.net/navigate-uk-telecoms/governm...,Open,Dorset Open Networks Ecosystem (DONE),NaN,None,UKTIN,...,GBP,Government Funded,2023-09-14 12:00:00,Open Networks Ecosystem Competition,86ec4e30-da2d-50eb-8b75-bbee7b10a03d,Dorset Council,"Dorset Council, ARM, Vodafone, Illuminate, Key...",3.619544,"MetadataReturn(creation_time=None, last_update...",telecoms
3,The ORanGaN project is developing a sovereign ...,NaT,2435528,None,https://uktin.net/navigate-uk-telecoms/governm...,Open,ORanGaN,NaN,None,UKTIN,...,GBP,Government Funded,2022-01-01 12:00:00,Part of the Future RAN Competition,809346f8-e260-5d00-a9b9-3a4611f19a37,Inex Microtechnology Limited,"Inex Microtechnology Limited, Custom Interconn...",3.619544,"MetadataReturn(creation_time=None, last_update...",telecoms
4,This scheme is intended to supplement the coun...,2025-03-31 12:00:00,250000,None,https://uktin.net/navigate-uk-telecoms/governm...,Open,Oxfordshire County Council,NaN,None,UKTIN,...,GBP,Government Funded,2023-10-02 12:00:00,Smart Infrastructure Pilots Programme,7c7a4ec9-c8b1-54c5-8331-12b716ae6b49,Oxfordshire County Council,"Oxfordshire County Council, Virgin Media O2",3.619544,"MetadataReturn(creation_time=None, last_update...",telecoms


In [6]:
# help(client.collections)

Try to read all objects
https://weaviate.io/developers/weaviate/manage-data/read-all-objects

In [7]:
# pprint.pprint(client.collections.list_all())

Which collection to use?

GTR_researchDatabaseAndModel

'GTR_researchMaterial'

'GTR_spinOut'

'UKTIN_lab'
'Universal_organisation'

'Universal_project'
'Universal_publication'

In [29]:
import csv 

def get_all_items(
    client,
    collection="Universal_project",
    datasources=["CORIS", "GTR"],
    max_items=1000,
    save=True,
    output_file="output.csv"
):
    """
    Get all items from selected datasources and save them to a .csv file
    """
    collection = client.collections.get(collection)
    
    # Open the CSV file in write mode
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write the header row
        writer.writerow(['id', 'name', 'source', 'url', 'description'])
        
        # Initialize a counter for the number of items processed
        count = 0
        
        # Iterate over the items in the collection
        for item in collection.iterator():
            properties = item.properties
            source = properties["dataSource"]
            
            if source not in datasources:
                print("not found")
                continue
            
            # Extract the desired fields
            item_id = str(item.uuid)
            name = properties["name"]
            url = properties["url"]
            description = properties["description"]
            
            # Write the item data to the CSV file
            writer.writerow([item_id, name, source, url, description])
            
            count += 1
            print(count)
            
            # Break the loop if the maximum number of items is reached
            if count >= max_items:
                break
        
        print(f"Processed {count} items and saved them to {output_file}")

get_all_items(client, save=True, output_file="data.csv")
# def get_all_items(
#         client, 
#         collection = "Universal_projects", 
#         datasources=["CORIS", "GTR"], 
#         max_items=1000, 
#         save=True
#     ):
#     """
#     Get all items from selected datasources and save them to a .csv file
#     """
#     collection = client.collections.get(collection)
    
#     # anything else needed here?

#     for n, item in enumerate(collection.items):
#         id = item.uuid
#         properties = item.properties
#         source = properties["source"]
#         if source not in datasources:
#             continue
#         else:
#             url = properties["url"]
#             name = properties["name"]
#             description = properties["description"]
#             # store the data: id, name, source, uri, description

1
not found
2
3
not found
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
not found
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
not found
68
69
70
71
72
73
74
75
76
77
not found
78
79
80
81
82
83
84
not found
85
86
87
88
not found
89
90
91
92
93
94
95
96
97
98
99
100
101
not found
102
103
104
105
106
107
108
not found
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
not found
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
not found
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
not found
182
183
184
185
186
187
188
189
190
191
192
193
not found
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
not found
228
229
230
231
232
233
not found
234
235
236
237
238
239
24

In [23]:
# Read all objects

collection = client.collections.get("Universal_project")
datasources = ["CORDIS", "GTR", "UKTIN"]
for n, item in enumerate(collection.iterator()):
    id = item.uuid
    properties = item.properties
    source = properties["dataSource"]
    url = properties["url"]
    name = properties["name"]
    if source not in datasources:
        pprint.pprint({"id": id, "datasource": properties["dataSource"]})
        continue
    if n > 1000000:
        break   

KeyboardInterrupt: 

In [17]:
# Read all objects

collection = client.collections.get("Universal_project")

for n, item in enumerate(collection.iterator()):
    pprint.pprint({"id": item.uuid, "properties": item.properties})
    if n > 10:
        break   

{'id': _WeaviateUUIDInt('00014afd-3c1f-410e-8d00-8ff5a7f7af54'),
 'properties': {'dataSource': 'GTR',
                'description': 'The search to understand the nature of the '
                               'elusive dark matter in our universe, making up '
                               '85% of its mass, is amongst the highest '
                               'scientific priorities around the world. '
                               'Terrestrial experiments focus efforts on '
                               'Weakly Interacting Massive Particles (WIMPs) '
                               'with ultra-low background experiments, '
                               'operating many tonnes of target mass in deep '
                               'underground sites. Such experiments have swept '
                               'the bulk of the available electroweak '
                               'parameter space for WIMPs and in the next '
                               'decade will approach an i